In [ ]:
from csv import reader
from math import sqrt
from random import seed
from random import randrange
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import time as time
import numpy as np

In [ ]:
input_data ='train.csv'

In [ ]:
df =pd.read_csv(input_data)

In [ ]:
print(df.shape)
df.head(n=5)

In [ ]:
df["label"].value_counts()

In [ ]:
df.sample(frac=1)

In [ ]:
df['label'].value_counts().plot.bar()

In [ ]:
data_X = df.loc[:,df.columns!= "label"].values.astype('float32')
data_Y =df[["label"]].values

data_X=np.array(data_X)
data_Y=np.array(data_Y)

data_X /= 255

data_X = data_X[:3500]
data_Y = data_Y[:3500]

In [ ]:
from sklearn.model_selection import train_test_split
from operator import itemgetter, attrgetter

X_t, X_test, Y_t, Y_test = train_test_split(data_X, data_Y,test_size=0.15,random_state=0)

In [ ]:
x = data_X.reshape(data_X.shape[0], 28, 28)
plt.figure(figsize=(8,8))
for i in range(15) :
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.subplot(3,5,i+1)
    plt.imshow(x[i],cmap=plt.cm.binary)
    plt.xlabel(data_Y[i])
plt.show()

In [ ]:
def euclidean_dist(given, row):
     dist = np.sum(np.square(np.subtract(given,row)))
     return np.sqrt(dist)

def getting_neighbours(train, test_row,number):
    distances = []
    for i in range(len(train)):
          dist = euclidean_dist(test_row,train[i])
          distances.append((i, dist))
    #print(distances)
    distances.sort(key=lambda tup: tup[1])
    # print(distances)
    all_neighbor = []
    for i in range(number):
        all_neighbor.append(distances[i][0])
    # print(all_neighbor)
    return all_neighbor

def predict_class(train, test_row,number):
    neighbors =getting_neighbours(train,test_row,number)
    # print(neighbors)
    classes= [int(Y_t[row]) for row in neighbors]
    # print(classes)
    class_c={}
    for x in range(len(classes)):
      v = classes[x]
      if v in class_c:
          class_c[v] += 1
      else:
          class_c[v] = 1
    sort_c = sorted(class_c.items(), key=lambda x: x[1], reverse=True)
    return sort_c[0][0]

def k_neighbors(train,test, number):
    pred =[]
    for row in test:
        neighb =predict_class(train,row,number)
        pred.append(neighb)
    return pred

def accuracy_pred(pred, test2):
  prediction_true = 0
  for i in range(len(pred)):
    if pred[i]==test2[i]:
      prediction_true += 1
  return (prediction_true/len(pred))*100.0

In [ ]:
def acc(number): # Main function 
  pred = k_neighbors(X_t,X_test,number)
  # print(pred)
  return pred

In [ ]:
from sklearn.metrics import confusion_matrix

start = time.time()
pred = acc(7)  #Main function call
model_acc = accuracy_pred(pred,Y_test)
end = time.time()
print("Model accuracy: "+str(model_acc))
print("Time taken: "+ str(end-start))

m =confusion_matrix(Y_test,pred)
p = sns.heatmap(pd.DataFrame(m), annot=True, cmap="Greens" ,fmt='g')
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')

In [ ]:
FN =(np.sum(m[1])-m[1][1]) # Sum of the numbers above the diagonal for the given digit( digit in the dark green square)
FP =(np.sum(m[:,1])-m[1][1]) # Sum of the numbers below the diagonal for the given digit( digit in the dark green square)
TP_TN = (np.sum(np.diagonal(m)))  # Sum of diagonal will give TP+TN

acc2 = TP_TN/(TP_TN + FP+FN)*100.0
print("Accuracy of number '1' is "+str(acc2))

In [ ]:
for x in range(3,20,4):
    start= time.time()
    pred =acc(x)
    model_acc= accuracy_pred(pred,Y_test)
    end =time.time()
    
    m =confusion_matrix(Y_test,pred)
    FN =(np.sum(m[1])-m[1][1]) 
    FP =(np.sum(m[:,1])-m[1][1]) 
    TP_TN = (np.sum(np.diagonal(m)))  
    
    acc2 = TP_TN/(TP_TN + FP+FN)*100.0
    print("Accuracy of number '1' for k = "+str(x)+" is: "+str(acc2))
    print("Time taken: "+ str(end-start))